# **Colab configuration**

In [ ]:
!wget https://github.com/ispc/ispc/releases/download/v1.18.0/ispc-v1.18.0-linux.tar.gz
!tar xvf ispc-v1.18.0-linux.tar.gz
!mkdir /content/sum_vector

# Sum of vectors elements: reduction algorithm

In [ ]:
%%writefile  /content/sum_vector/sum_vector.cpp
#include <stdio.h>
#include <stdlib.h>

#include "sum_vector_ispc.h"
#include "timing.h"

#define NDATA (1<<15)

using namespace ispc;

int main() {
    double vin[NDATA], res=0;

    // Initialize input buffer
    for (int i = 0; i < NDATA; ++i)
        vin[i] = (double)i;
    reset_and_start_timer();
    for (int i = 0; i < NDATA; ++i)
        res += vin[i];
    double tser = get_elapsed_mcycles();
    printf("@time of serial run:\t\t\t\t[%.3f] million cycles\n", tser);
    printf("res = %f\n", res);

    // Call simple() function from simple.ispc file
    reset_and_start_timer();
    res = sum_vector(vin, NDATA);
    double tpar = get_elapsed_mcycles();
    printf("@time of parallel run:\t\t\t\t[%.3f] million cycles\n", tpar);
    printf("@speedup=%f\n", tser/tpar);
    printf("res = %f\n", res);

    return 0;
}

In [ ]:
%%writefile  /content/sum_vector/sum_vector.ispc
export uniform double sum_vector(uniform double A[], uniform int n)
{
  double sum = 0;
  foreach (i = 0 ... n)
        sum += A[i];
  return reduce_add(sum);
}



# Code compilation with ISPC
Documentation on ISPC compiler can be found at: [link to ISPC](https://ispc.github.io/index.html)

In [ ]:
%%writefile  /content/sum_vector/script.sh
#!/bin/bash

rm -f /content/sum_vector/sum_vector_ispc.h
/content/ispc-v1.18.0-linux/bin/ispc /content/sum_vector/sum_vector.ispc -o /content/sum_vector/sum_vector.obj -h /content/sum_vector/sum_vector_ispc.h
clang++ -o /content/sum_vector/sum_vector /content/sum_vector/sum_vector.cpp -O0 /content/sum_vector/sum_vector.obj -I /content/sum_vector/ -I /content/ispc-v1.18.0-linux/examples/common/

In [ ]:
!cd /content/sum_vector/
!bash /content/sum_vector/script.sh

# Code execution

In [ ]:
!/content/sum_vector/sum_vector